# PRVI ZADATAK - Viterbijev algoritam u log-spaceu

Eleonora Detić, Eva Belcar

Izradite Viterbijev algoritam (za povremeno nepoštenu kockarnicu) u log-space-u. Drugim riječima, umjesto s produktima (vjerojatnosti), baratat ćemo sa sumama (logaritama vjerojatnosti).
Kod testirajte na nekom dovoljno dugačkom nizu (barem 1000 znakova).
Parametre modela uzmite iz prethodnog zadatka o kockarnici.

In [1]:
import random
import numpy as np
#import os
import numpy as np
#import ViterbiAlgorithm
#from tabulate import tabulate
from generate_observations import generate_observations
#from generate_observations import print_result
from estimate_based_on_observation import pretty_print
#from estimate_based_on_observation import updateMatricesAfterRun
from estimate_based_on_observation import runLoop

Generiramo random uzorak od 1000 brojeva.
Ne uzimamo uniformnu distribuciju jer će nam tada ispasti da se uvijek nalazimo u _fair_ kockici, pa simuliramo uzorak s _wighted_ parametrima (sami odabiremo parametre).

In [2]:
randomlist = random.choices(['1','2','3','4','5','6'], [0.1,0.1,0.1,0.1,0.1,0.3], k = 1000)
x = "".join([str(int) for int in randomlist])

Matricu $V$ računamo rekurzivno prema formuli:
$$V_k(i)=e_k(i)\max_{j\in\{0,1\}} \left(a_{jk}V_j(i-1)\right)\to \boxed{\ln V_k(i)=\ln e_k(i)+ \max_{j\in\{0,1\}} \left(\ln a_{jk}+ \ln V_j(i-1)\right)}$$
gdje je $j,k\in\{0,1\}$, $a_{00}=cl$, $a_{01}=dl$, $a_{10}=df$ i $a_{11}=cf$, dok je $i$ broj koji smo dobili u tom bacanju.

Treba paziti da ne logaritmiramo više puta.

In [3]:
def new_viterbi(F,L,cl,dl,cf,df):
    tmp = []
    V = []
    for i in range(len(x)):
        tmp.append(0) #array koji sadrži nule, iste duljine kao i x

    for i in range(2):
        V.append(tmp[:]) #prazna matrica 2 puta duljina od x


    k = int(x[0]) - 1 


    V[0][0] = F[k] #vjerojatnost da je pao prvi broj (iz generiranog niza) ako sam u fair kockici
    V[1][0] = L[k] #vjerojatnost da je pao prvi broj (iz generiranog niza) ako sam u floaded kockici


#rekurzivno računamo V pomoću gore navedene formule
    for i in range(1,  len(x)):
        tmp = []                         #privremena lista
        k = int(x[i]) - 1                #dohvaćam indeks broja koji je pao u i-tom bacanju
        tmp.append(V[0][i-1]+F[k]+cl)    #appendam privremenu listu: log-vjer. ako smo u prošlom bili u F
                                                                # + log-vjer. da je pao broj (k+1) u F kocki
                                                                # + log-vjer. da cu ostat u L kocki?
        tmp.append(V[1][i-1]+F[k]+df)    #appendam privremenu listu: log-vjer. ako smo u prošlom bili u L
                                                                # + log-vjer. da je pao broj (k+1) u F kocki
                                                                # + log-vjer. da cu preći iz F u L?
        tmp.sort()                       #sortiram privremenu listu
        V[0][i] = tmp[1]                 #biram prvi element (maksimalni) od ta dva
    
    #ponavljam postupak od gore
        tmp = []
        tmp.append(V[1][i-1]+L[k]+cf)
        tmp.append(V[0][i-1]+L[k]+dl)
        tmp.sort()
        V[1][i] = tmp[1]    
    
    
#ako mi je zadnja log-vjer. u matrici V veća u prvom redu (fair), postavi zastavicu na nula, inače 1
    if V[0][len(V[0])-1] > V[1][len(V[1])-1]:
        l = 0
    else:
        l = 1


    le = len(V[0]) - 1    #duljina "preostalog tracebacka"
    put = ''              #spremanje optimalnog puta

    while le >= 0:
        
        k = int(x[le])-1              #dohvaćam indeks(potrebno za emisijsku matricu)
        
        if le == 0:                   #slučaj kada dođemo do zadnjeg elementa
            if l == 1:                #bili smo u loaded kockici
                put = 'L' + put  
                le = le - 1
            elif l == 0:              #inače smo bili u fair kockici
                put = 'F' + put 
                le = le - 1
          
        elif l == 0:            #s ovim cudnim nejednakostima provjeravam jednakost? Greška je zaokruživanje?
                                #sigurno ću dodati F u put jer l=0
                                #provjeravam hoću li ostati u F ili prelazim u L (traceback po formuli)
            if (( V[l][le]/( V[l+1][le-1]+(F[k]+df) ) ) > 0.99999999999 and ( V[l][le]/( V[l+1][le-1]+(F[k]+df) ) )<=1.0000000000000003 ):
                put = 'F' + put 
                le = le - 1
                l = l + 1
                                  #po formuli zbog df, znači da je ovo stanje proizašlo iz loaded kockice (l postaje 1)
                                  #iz L u F
                                  #analogno slijedi za ostale slučajeve
            elif ( ( V[l][le]/( V[l][le-1]+(F[k]+cl) ) ) > 0.9999999999 and ( V[l][le]/( V[l][le-1]+(F[k]+cl) ) )<=1.0000000000000003 ):
                put = 'F' + put
                le = le - 1
    
        elif l == 1:
            
            if ( ( V[l][le]/( V[l-1][le-1]+(L[k]+dl) ) ) > 0.99999999999 and ( V[l][le]/( V[l-1][le-1]+(L[k]+dl) ) )<=1.0000000000000003 ):
                put = 'L' + put  
                le = le - 1
                l = l - 1
                
            elif ( ( V[l][le]/( V[l][le-1]+(L[k]+cf) ) ) > 0.999999999 and ( V[l][le]/( V[l][le-1]+(L[k]+cf) ) )<=1.0000000000000003 ):
                put = 'L' + put
                le = le - 1        
    return(put)   
    

In [4]:
#emisijska matrica
F = np.log10([ 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0 ]) #FAIR
L = np.log10([ 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/2.0]) #LOADED

#tranzicijska matrica
cl = np.log10(0.9)
dl = np.log10(0.1)
cf = np.log10(0.8)
df = np.log10(0.2)

new_viterbi(F,L,cl,dl,cf,df)

'LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLFFFFFFFFFLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLFFFFFFFFFFFLLLLLLLFFFFFFFFFFFFFFFFFLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLFFFFFFFFLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL

## Zadatak 2

U narednim zadacima cemo se baviti optimizacijom modela, bolje receno, optimizacijom parametara modela; ako imamo kockarnicu sa samo jednom kockom i dani niz - oznacimo ga sa D (data) - onda je jasno koji su parametri θ
θkoji maksimiziraju vjerodostojnost podataka, tj.

(dokazite, tj. pronadjite dokaz); no, ako se nas model sastoji od barem dvije kocke, ne postoji zatvorena formula kojom bismo odredili optimalne parametre; zbog toga se sluzimo raznim aproksimacijama i/ili iterativnim procedurama da "pronadjemo" optimalne parametre; u ovom cemo zadatku napraviti dio koraka za iterativnu/aproksimativnu optimizaciju parametara modela (pomocu Viterbijevog algoritma); dakle, zadatak je:

i) simulirati dugacak niz (5000 ili 10000 znakova, recimo) iz nekog modela (s dvije kocke)

ii) za neki model, s nekim drugim parametrima (razlicitim, ali ne jako razlicitim), pronaci optimalan put/prolaz niza kroz model

iii) za tako dobiveni prolaz, odrediti emisijske i tranzicijske frekvencije (npr. kocka 1 je znak 1 emitirala 891 put, znak 2 902 puta, itd...., analogno za tranzicije); nakon normalizacije (tako da se odgovarajuci parametri sumiraju na 1), imate "nove" parametre modela

iv) provjerite koliko su vam tako dobiveni "novi" parametri slicni pocetnima (onima iz kojih ste simulirali)

In [8]:
#zadajemo prvi model sljedećim parametrima
initial_state_probability = [0.9999, 0.0001]
transition_matrix = [[0.95, 0.05],\
                        [0.05, 0.95]]
emission_matrix = [[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],\
                      [0.12, 0.12, 0.12, 0.12, 0.12, 0.4]]
state_space=['F','L']
observation_space=['1', '2', '3', '4', '5', '6']
print("Truth:")
pretty_print(initial_state_probability, transition_matrix, emission_matrix, state_space, observation_space)

#za taj prvi model generiramo niz, i) dio zadatka
#ova funkcija mi vraca prava stanja(jesam li u fair ili loaded kocki)
#također vraća niz dobivenih brojeva
true_state, dice_rolls = generate_observations(initial_state_probability, transition_matrix, emission_matrix,
                                        num_samples=5000, #radim za 5000 jer mi je tak brže
                                        state_space=state_space,
                                        observation_space=observation_space,
                                        seed=515)

#mogu ispisati ovdje
#print("Simulirani niz", "".join([str(int) for int in dice_rolls]))


# ii) uzimam drugi model, razlicit ali ne jako razlicit 
estimated_transition_matrix = [[0.80, 0.20],\
                                [0.20, 0.80]]
estimated_emission_matrix = [[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],\
                                 [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]]

#ovdje procijenim parametre tranzicijske i empirijske matrice 
estimated_transition_matrix, estimated_emission_matrix = \
    runLoop(initial_state_probability, estimated_transition_matrix, estimated_emission_matrix, 
            dice_rolls,
            state_space=state_space,
            observation_space=observation_space,
            runs=15)

print("\nEstimated: ")
pretty_print(initial_state_probability, estimated_transition_matrix, estimated_emission_matrix, state_space, observation_space)

Truth:
| Transitions   |      F |      L |
|---------------|--------|--------|
| Begin         | 0.9999 | 0.0001 |

| Transitions   |    F |    L |
|---------------|------|------|
| F             | 0.95 | 0.05 |
| L             | 0.05 | 0.95 |

| Emissions   |        1 |        2 |        3 |        4 |        5 |        6 |
|-------------|----------|----------|----------|----------|----------|----------|
| F           | 0.166667 | 0.166667 | 0.166667 | 0.166667 | 0.166667 | 0.166667 |
| L           | 0.12     | 0.12     | 0.12     | 0.12     | 0.12     | 0.4      |

Run 1 Probability: -9714.920318254792
Run 2 Probability: -9019.840906483256
Run 3 Probability: -8884.103759877164
Run 4 Probability: -8820.800068208806
Run 5 Probability: -8810.965491632382
Run 6 Probability: -8802.896008039013
Run 7 Probability: -8794.294434078467
Run 8 Probability: -8783.806480518755
Run 9 Probability: -8777.851586866054
Run 10 Probability: -8777.835811070765
Run 11 Probability: -8777.835811070765
Conver